In [1]:
from bs4 import BeautifulSoup
import requests
import sys

# Method to read new by new
def getData(soup, key):
    for article in soup.findAll('article'):
        # Initialize Variables
        titulo = ''
        autor = ''
        titulo_busqueda = ''
        etiqueta = ''
        fecha = ''
        hora = ''
        titulo_imagen = ''
        imagen = ''
        text = ''

        # Iterate Articles
        for div in article.findAll('div'):
            if 'news__media' in div['class']:
                imagen = div.find('img').get('data-lazy-src').strip()
                titulo_imagen = div.find('a').get('title').strip()
            else :
                data = div.findAll('a')
                if len(data)>=2 :
                    etiqueta = data[0].get_text().strip()
                    titulo_busqueda = data[1].get_text().strip()
                    link = data[1].get('href')
                else :
                    etiqueta = 'S/E'
                    titulo_busqueda = data[0].get_text().strip()
                    link = data[0].get('href')

                page2 = requests.get(link)
                soup2 = BeautifulSoup(page2.content)

                # Get news content
                if soup2.find('h1') :
                    titulo = soup2.find('h1').getText().strip()
                else :
                    continue

                if soup2.find('time'):
                    dt = soup2.find('time').getText().split()
                    hora = (dt[-5] + '' + dt[-4]).replace('ET','').replace('(','').replace('GMT)','')
                    fecha = (dt[-3] + ' ' + dt[-2] + ' ' + dt[-1]).replace(',','')
                else :
                    hora = 'S/H'
                    fecha = 'S/F'

                if soup2.find('div', class_='storyfull__info') :
                    autor = soup2.find('div', class_='storyfull__info').find('a').getText() if soup2.find('div', class_='storyfull__info').find('a') else 'S/A'
                elif soup2.find('p', class_='news__byline') :
                        autor = soup2.find('p', class_='news__byline').find('span').find('span').getText() if soup2.find('p', class_='news__byline').find('span') else 'S/A'
                else :
                    autor = 'S/A'

                if soup2.find('div', class_='col--main') :
                    bodyText = soup2.find('div', class_='col--main')
                else :
                    continue

                if bodyText.find('div', class_='storyfull__body') :
                    for p in bodyText.find('div', class_='storyfull__body').findAll('p') :
                        text += p.getText().replace(';','').strip().rstrip()
                elif bodyText.find('div', class_='news__excerpt') :
                    for p in bodyText.find('div', class_='news__excerpt').findAll('p') :
                     text +=  p.getText().replace(';','').strip().rstrip()
                else :
                    text = 'S/T'

        # write Row
        newLine = key + ';' + titulo + ';' + autor + ';' + titulo_busqueda + ';' + etiqueta + ';' + fecha + ';' + hora + ';' + titulo_imagen + ';' + imagen + ';' + link + ';' + text + '\n'
        newLine=newLine.encode(sys.stdout.encoding, errors='replace')
        with open("news_pandemia_covid19.csv", "ab") as myfile:
            myfile.write(newLine)
            myfile.close()
    return


# Write Headers
newLine = 'Clave;Titulo;Autor;Titulo Busqueda;Etiqueta;Fecha;Hora;Titulo Imagen;Imagen URL;Noticia URL;Contenido\n'
newLine = newLine.encode(sys.stdout.encoding, errors='replace')
with open("news_pandemia_covid19.csv", "ab") as myfile:
    myfile.write(newLine)
    myfile.close()

# List of words/keys to search
listBusqueda = ['covid-19', 'coronavirus', 'sars-cov-2', 'vacuna', 'pandemia']

# For to Iterate List of Words
for a in listBusqueda :
    key = a
    search = requests.get("https://cnnespanol.cnn.com/?s=" + key)
    soup = BeautifulSoup( search.content )

    # Search div of pagination
    divMain = soup.find('div', class_= 'col--main')
    pagination = divMain.find('div', 'search__wrappagination' )

    # Get total pages of search
    pages = 0
    for pageNum in pagination.findAll('a', class_='page-numbers') :
        if len(pageNum["class"]) != 1:
             continue;
        nextPage = int(pageNum.getText())
        pages = nextPage if pages<nextPage else pages

    # Iterate Pages
    for page in range(1,pages) :
        # Concatenate Page and Key
        pageSearch = requests.get("https://cnnespanol.cnn.com/page/" + str(page) + "/?s=" + key)
        soup2 = BeautifulSoup( pageSearch.content )

        # If there are not more articles finish iteration
        if soup2.findAll('article') :
            # Method to read articles
            getData(soup2, key)
        else :
            break
